In [1]:
import polars as pl
import polars.selectors as cs
import gzip

In [2]:
df = pl.read_csv('~/buckets/b1/datasets/competencia_02.csv.gz', infer_schema_length=100000, dtypes={'foto_mes':str})

In [3]:
columns = df.columns 
columns.remove('foto_mes')
columns.remove('numero_de_cliente')
columns.remove('clase_ternaria')

In [8]:
df = df.sort('foto_mes')

In [9]:
max_lag = 6
numeric_columns = list(df.select(cs.numeric()).columns)
numeric_columns.remove('numero_de_cliente')

expressions = list([
    pl.col(column).shift(lag).over('numero_de_cliente').alias(column + '_lag_' + str(lag)) 
        for column in columns for lag in range(1, max_lag + 1)])

df = df.with_columns(expressions)

In [ ]:
expressions=list((pl.col(column) - pl.col(column + '_lag_1')).alias(column + "_diff_1") for column in numeric_columns)

expressions.extend([
    (
        pl.col(column + '_lag_' + str(lag)) - pl.col(column + '_lag_' + str(lag+1))        
    ).alias(column + '_diff_' + str(lag+1)) for column in numeric_columns for lag in range(1, max_lag)])

df = df.with_columns(expressions)

In [11]:
# Save the result as gzip
with gzip.open('~/buckets/b1/datasets/competencia_02_6lags.csv.gz', 'wb') as f:
    df.write_csv(f)
